In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

# Fine-tune Whisper

## Prepare Data and Model

### Load Model

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

In [ ]:
# include forced token in the training
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
# to use gradient checkpointing
model.config.use_cache = False

### Load Datasets

In [2]:
AUDIO_COLUMN_NAME = "audio"
TEXT_COLUMN_NAME = "sentence"

In [3]:
from datasets import Audio, DatasetDict, concatenate_datasets, load_dataset

def normalize_dataset(ds, audio_column_name=None, text_column_name=None):
    if audio_column_name is not None and audio_column_name != AUDIO_COLUMN_NAME:
        ds = ds.rename_column(audio_column_name, AUDIO_COLUMN_NAME)
    if text_column_name is not None and text_column_name != TEXT_COLUMN_NAME:
        ds = ds.rename_column(text_column_name, TEXT_COLUMN_NAME)
    # resample to the same sampling rate
    ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
    # normalise columns to ["audio", "sentence"]
    ds = ds.remove_columns(set(ds.features.keys()) - set([AUDIO_COLUMN_NAME, TEXT_COLUMN_NAME]))
    return ds

In [4]:
raw_datasets = DatasetDict()

ds_train_mcv = load_dataset("mozilla-foundation/common_voice_11_0", "fr", split="train+validation", use_auth_token=True)
ds_train_mcv = normalize_dataset(ds_train_mcv)

ds_train_mls = load_dataset("facebook/multilingual_librispeech", "french", split="train+validation")
ds_train_mls = normalize_dataset(ds_train_mls, text_column_name="text")

raw_datasets["train"] = concatenate_datasets([ds_train_mcv, ds_train_mls])

# NB: shuffle concatenated dataset
raw_datasets["train"] = raw_datasets["train"].shuffle(seed=10)

raw_datasets["eval"] = load_dataset("mozilla-foundation/common_voice_11_0", "fr", split="test", use_auth_token=True)
raw_datasets["eval"] = normalize_dataset(raw_datasets["eval"])

raw_datasets

/home/bhuang/anaconda3/envs/jpt/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Reusing dataset common_voice_11_0 (/home/bhuang/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/fr/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f)
Reusing dataset multilingual_librispeech (/home/bhuang/.cache/huggingface/datasets/facebook___multilingual_librispeech/french/2.1.0/1904af50f57a5c370c9364cc337699cfe496d4e9edcae6648a96be23086362d0)
/home/bhuang/anaconda3/envs/jpt/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Reusing dataset common_voice_11_0 (/home/bhuang/.cache/huggingface/datasets/mozilla-foundation___common

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 761752
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 16089
    })
})

In [5]:
# only use part of training and test set for demonstration
# comment this
num_train_examples = 500
num_test_examples = 100

raw_datasets["train"] = raw_datasets["train"].select(range(num_train_examples))
raw_datasets["eval"] = raw_datasets["eval"].select(range(num_test_examples))

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 500
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 100
    })
})

### Data Augmentation

In [6]:
# download musan dataset for background noise
# ! wget https://www.openslr.org/resources/17/musan.tar.gz
# ! tar -zxvf musan.tar.gz
# ! rm -r ./musan/speech

In [7]:
from audiomentations import (
    AddBackgroundNoise,
    AddGaussianNoise,
    Compose,
    Gain,
    OneOf,
    PitchShift,
    PolarityInversion,
    TimeStretch,
)

musan_dir = "./musan"

# define augmentation
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=0.2, leave_length_unchanged=False),
        Gain(min_gain_in_db=-6, max_gain_in_db=6, p=0.1),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.2),
        OneOf(
            [
                AddBackgroundNoise(
                    sounds_path=musan_dir, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0
                ),
                AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=1.0),
            ],
            p=0.2,
        ),
    ]
)


def augment_dataset(batch):
    # load and (possibly) resample audio data to 16kHz
    sample = batch[AUDIO_COLUMN_NAME]

    # apply augmentation
    augmented_waveform = augmentation(sample["array"], sample_rate=sample["sampling_rate"])
    batch[AUDIO_COLUMN_NAME]["array"] = augmented_waveform
    return batch

In [8]:
preprocessing_num_workers = 4

In [9]:
# augment training data
augmented_raw_training_dataset = raw_datasets["train"].map(
    augment_dataset, num_proc=preprocessing_num_workers, desc="augment train dataset"
)

# combine
raw_datasets["train"] = concatenate_datasets([raw_datasets["train"], augmented_raw_training_dataset])
raw_datasets["train"] = raw_datasets["train"].shuffle(seed=10)

raw_datasets

Parameter 'function'=<function augment_dataset at 0x7f6083ea0f70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


augment train dataset #0:   0%|          | 0/125 [00:00<?, ?ex/s]

augment train dataset #1:   0%|          | 0/125 [00:00<?, ?ex/s]

augment train dataset #2:   0%|          | 0/125 [00:00<?, ?ex/s]

augment train dataset #3:   0%|          | 0/125 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1000
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 100
    })
})

### Preprocessing

In [10]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

In [11]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="french", task="transcribe")

In [12]:
do_normalize_text = True


def prepare_dataset(batch):
    # load
    audio = batch[AUDIO_COLUMN_NAME]
    # compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[
        0
    ]
    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    # process targets
    input_str = normalizer(batch[TEXT_COLUMN_NAME]).strip() if do_normalize_text else batch[TEXT_COLUMN_NAME]
    # encode target text to label ids
    batch["labels"] = processor.tokenizer(input_str).input_ids

    return batch

In [ ]:
vectorized_datasets = raw_datasets.map(
    prepare_dataset, remove_columns=next(iter(raw_datasets.values())).column_names, desc="preprocess dataset"
)

preprocess dataset:   0%|          | 0/1000 [00:00<?, ?ex/s]

### Remove Long Audio

In [ ]:
max_input_length = 30
min_input_length = 0


def is_audio_in_length_range(length):
    return length > min_input_length and length < max_input_length

In [ ]:
vectorized_datasets = vectorized_datasets.filter(
    is_audio_in_length_range, num_proc=preprocessing_num_workers, input_columns=["input_length"]
)

vectorized_datasets

### Remove Long Text

In [ ]:
max_label_length = model.config.max_length
max_label_length

In [ ]:
def is_labels_in_length_range(labels):
    return len(labels) < max_label_length

In [ ]:
vectorized_datasets = vectorized_datasets.filter(
    is_labels_in_length_range, num_proc=preprocessing_num_workers, input_columns=["labels"]
)

vectorized_datasets

## Training and Evaluation

### Data Collator

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import torch


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        # convert to tensors
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad label ids to the max length in the batch
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
# evaluate with the 'normalised' WER
do_normalize_eval = True


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        # perhaps already normalised
        label_str = [normalizer(label) for label in label_str]
        # filtering step to only evaluate the samples that correspond to non-zero references
        pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
        label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

    wer = metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Training Configuration

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./outputs/whisper_medium_ft",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    warmup_steps=800,
    max_steps=8000,
    learning_rate=6.25e-6,
    weight_decay=0.01,
    gradient_checkpointing=True,
    fp16=True,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=25,
    report_to=["tensorboard"],
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

### Initialize Trainer

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets["eval"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

### Training

In [ ]:
train_result = trainer.train()

In [ ]:
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

## Evaluation

In [ ]:
eval_metrics = trainer.evaluate(metric_key_prefix="eval")

eval_metrics